#### 케라스로 LeNet 구현하기

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Dense
from tensorflow.keras import backend as K
 
class LeNet:

    @staticmethod 
    def build(numChannels, imgRows, imgCols, numClasses, 
              activation="relu"):

        model = Sequential()
        inputShape = (imgRows, imgCols, numChannels)
        # "channels first"이면 shape를 변경함 
        if K.image_data_format() == "channels_first":
            inputShape = (numChannels, imgRows, imgCols)
        # 첫 번째 CONV => ACTIVATION => POOL 계층 
        model.add(Conv2D(6, 5, padding="same", input_shape=inputShape))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        # 두 번째 CONV => ACTIVATION => POOL 계층 
        model.add(Conv2D(16, 5, padding="same"))
        model.add(Activation(activation))
        model.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
        
        # 첫 번째 FC => ACTIVATION 계층 
        model.add(Flatten())
        model.add(Dense(120))
        model.add(Activation(activation))
        # 두 번째 FC 계층 
        model.add(Flatten())
        model.add(Dense(84))
        model.add(Activation(activation))
        # 출력층 soft-max 활성화 함수 사용
        model.add(Dense(numClasses))
        model.add(Activation("softmax"))

        return model

#### LeNet을 이용한 MNIST 숫자이미지 분류

In [2]:
# import the necessary packages
from tensorflow.keras.datasets import mnist
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import utils
import numpy as np
import cv2

In [3]:
((trainData, trainLabels), (testData, testLabels)) = mnist.load_data()
 
if K.image_data_format() == "channels_first":
    trainData = trainData.reshape((-1, 1, 28, 28))
    testData = testData.reshape((-1, 1, 28, 28))
else:
    trainData = trainData.reshape((-1, 28, 28, 1))
    testData = testData.reshape((-1, 28, 28, 1))
# [0, 1] Scaling 
trainData = trainData.astype("float32") / 255.0
testData = testData.astype("float32") / 255.0

In [4]:
trainLabels = utils.to_categorical(trainLabels, 10)
testLabels = utils.to_categorical(testLabels, 10)

In [5]:
opt = SGD(learning_rate=0.01)
model = LeNet.build(numChannels=1, imgRows=28, imgCols=28,numClasses=10)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])

In [6]:
model.fit(trainData, trainLabels, batch_size=200, epochs=30)

Epoch 1/30
300/300 [==============================] - 4s 13ms/step - loss: 1.9823 - accuracy: 0.3328
Epoch 2/30
300/300 [==============================] - 4s 13ms/step - loss: 0.4195 - accuracy: 0.8711
Epoch 3/30
300/300 [==============================] - 4s 13ms/step - loss: 0.2769 - accuracy: 0.9166
Epoch 4/30
300/300 [==============================] - 4s 13ms/step - loss: 0.2209 - accuracy: 0.9314
Epoch 5/30
300/300 [==============================] - 4s 13ms/step - loss: 0.1861 - accuracy: 0.9433
Epoch 6/30
300/300 [==============================] - 4s 13ms/step - loss: 0.1597 - accuracy: 0.9532
Epoch 7/30
300/300 [==============================] - 4s 13ms/step - loss: 0.1508 - accuracy: 0.9548
Epoch 8/30
300/300 [==============================] - 4s 13ms/step - loss: 0.1337 - accuracy: 0.9595
Epoch 9/30
300/300 [==============================] - 4s 13ms/step - loss: 0.1148 - accuracy: 0.9662
Epoch 10/30
300/300 [==============================] - 4s 13ms/step - loss: 0.1083 - accura

In [7]:
(loss, accuracy) = model.evaluate(testData, testLabels, batch_size=128, verbose=1)
print("accuracy: {:.2f}%".format(accuracy * 100))

79/79 [==============================] - 0s 2ms/step - loss: 0.0515 - accuracy: 0.9830
accuracy: 98.30%


In [8]:
# check to see if the model should be saved to file
model.save_weights("lenet_weights.hdf5", overwrite=True)

In [9]:
samples = np.random.choice(np.arange(0,len(testLabels)),size=(10,))

for i in samples:
    probs = model.predict(testData[np.newaxis, i])
    prediction = probs.argmax(axis=1)

    if K.image_data_format() == "channels_first":
        image = (testData[i][0] * 255).astype("uint8")
    else:
        image = (testData[i] * 255).astype("uint8")
 
    image = cv2.merge([image] * 3)
 
    image = cv2.resize(image, (96, 96),
                       interpolation=cv2.INTER_LINEAR)
 
    cv2.putText(image, str(prediction[0]), (5, 20),
                cv2.FONT_HERSHEY_SIMPLEX, 0.75, (0,255,0), 2)
    print("Predicted: {}, Actual: {}".format(prediction[0],
                                             np.argmax(testLabels[i])))
    cv2.imshow("Digit", image)
    cv2.waitKey(0)

cv2.destroyAllWindows()

Predicted: 4, Actual: 4
Predicted: 0, Actual: 0
Predicted: 1, Actual: 1
Predicted: 3, Actual: 3
Predicted: 3, Actual: 3
Predicted: 7, Actual: 7
Predicted: 4, Actual: 4
Predicted: 0, Actual: 0
Predicted: 4, Actual: 4
Predicted: 9, Actual: 9
